In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as report

data="process.csv"

In [6]:
df = pd.read_csv(data)
df = df.fillna(-2)
df.head()


,Shape,formula_pretty,formula_anonymous,volume,density,density_atomic,energy_per_atom,band_gap,efermi,is_gap_direct,is_metal,is_magnetic,ordering,total_magnetization,theoretical
0,0,AlPbO3,ABC3,56.379268,8.311042,11.275854,-6.651740,0.000,6.457979,0,1,0,0,0.032580,1
1,2,Ba2CoMoO6,ABC2D6,277.167172,6.296940,13.858359,-7.604548,1.274,3.062452,1,0,1,2,0.000002,1
2,0,Ba2CoMoO6,ABC2D6,276.766068,6.306065,13.838303,-7.613764,0.000,3.727364,0,1,1,2,0.010982,0
3,4,Ba3HoIrRuO9,ABCD3E9,448.998994,7.501607,14.966633,-7.461055,0.000,3.778487,0,1,1,1,4.000589,1
4,4,Ba3LiOs2O9,AB2C3D9,428.639080,7.309243,14.287969,-7.343171,0.000,3.513875,0,1,1,1,6.011189,0


In [7]:
features=['volume', 'density', 'density_atomic', 'energy_per_atom','band_gap', 'efermi','is_gap_direct', 'is_metal', 'is_magnetic', 'ordering','total_magnetization','theoretical']  
X = df.loc[0:len(df), features].values
Y = df.loc[0:len(df), "Shape"].values

In [8]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.15, random_state=42)
print(len(X_train),len(X_test))
oversample = SMOTE()#oversampling of data
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

12344 2179


In [9]:
def CRandCM(method,X_test,Y_test):#a function to print classification report and confusion matrix
    Y_pred=method.predict(X_test)
    cr = report(Y_test,  Y_pred)
    cm = confusion_matrix(Y_test, Y_pred)
    print(cr)
    print(cm)

In [16]:
param_grid = {
    'n_estimators': [250,300, 200],
    'max_depth': [50, 100, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}
rfc = RandomForestClassifier()

grid_search = GridSearchCV(
    rfc, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs=-1, 
    scoring='accuracy', 
    verbose=1
)
grid_search.fit(X_train, Y_train)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [50, 100, None],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [250, 300, 200]},
             scoring='accuracy', verbose=1)

In [17]:
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 2, 'n_estimators': 250}


In [18]:
rfc = RandomForestClassifier(**grid_search.best_params_)
rfc.fit(X_train, Y_train)
CRandCM(rfc,X_test,Y_test)


              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1000
           1       0.91      0.94      0.93       362
           2       0.94      0.82      0.87       148
           3       0.94      0.90      0.92       115
           4       0.95      0.90      0.92        39
           5       0.91      0.88      0.90       300
           6       0.89      0.94      0.92       215

    accuracy                           0.94      2179
   macro avg       0.93      0.91      0.92      2179
weighted avg       0.94      0.94      0.94      2179

[[987   7   3   0   0   1   2]
 [  3 341   3   0   1  11   3]
 [ 11   3 121   5   1   4   3]
 [  6   3   0 103   0   1   2]
 [  0   3   0   0  35   1   0]
 [  6  13   2   1   0 264  14]
 [  0   5   0   1   0   7 202]]


In [19]:
accuracy = rfc.score(X_test, Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9421753097751262
